In [5]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
    return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [6]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

def gradf(a, b, c):
  df_da = -3*a**2 - 0.5*a**(-0.5)
  df_db = 3*cos(3*b)
  df_dc = 1/(c**2)
  return [df_da, df_db, df_dc] # todo, return [df/da, df/db, df/dc]

# expected answer is the list of 
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
WRONG! for dim 1: expected 10.25699027111255, yours returns -2.7333907856540307
OK for dim 2: expected 0.0625, yours returns 0.0625


In [7]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell


a, b, c = 2, 3, 4

# let h (small perturbation)
h = 0.000001
# -----------
numerical_grad = [] # TODO
# -----------
for i, var in enumerate([a, b, c]):
    inputs = [a, b, c]
    inputs[i] += h # add h to the i-th variable
    L1 = f(*inputs)
    L2 = f(a, b, c)
    grad_i = (L1 - L2) / h
    numerical_grad.append(grad_i)





for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353559348809995
OK for dim 1: expected 10.25699027111255, yours returns 10.256991666679482
OK for dim 2: expected 0.0625, yours returns 0.062499984743169534


In [8]:
# there is an alternative formula that provides a much better numerical 
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.
a, b, c = 2, 3, 4

# let h (small perturbation)
h = 0.000001


# -----------
numerical_grad2 = [] # TODO
# -----------
for i, var in enumerate([a, b, c]):
    inputs_pos = [a, b, c]
    inputs_neg = [a, b, c]

    inputs_pos[i] +=h
    inputs_neg[i] -=h

    L3 = f(*inputs_pos)
    L4 = f(*inputs_neg)

    grad_i = (L3-L4) / (2*h)    # symmetric derivative formula
    numerical_grad2.append(grad_i)
    
for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")

OK for dim 0: expected -12.353553390593273, yours returns -12.353553391353245
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027401572
OK for dim 2: expected 0.0625, yours returns 0.06250000028629188


In [9]:
from math import exp as math_exp, log as math_log

class Value:
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label

    def __repr__(self):
        return f"Value(data={self.data})"

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward

        return out

    def __sub__(self, other):
        return self + (-other)

    def __neg__(self):
        return self * -1

    def __truediv__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        return self * other**-1

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out

    def __pow__(self, power):
        assert isinstance(power, (int, float))
        out = Value(self.data ** power, (self,), f'**{power}')

        def _backward():
            self.grad += (power * self.data ** (power - 1)) * out.grad
        out._backward = _backward
        return out

    def exp(self):
        out = Value(math_exp(self.data), (self,), 'exp')

        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward
        return out

    def log(self):
        out = Value(math_log(self.data), (self,), 'log')

        def _backward():
            self.grad += (1 / self.data) * out.grad
        out._backward = _backward
        return out

    def __radd__(self, other):
        return self + other  # that's a kick. it was the main error. >> representing sum(counts) at denominator

    def backward(self):
        topo = []
        visited = set()

        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)

        build_topo(self)

        self.grad = 1.0
        for node in reversed(topo):
            node._backward()


In [10]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")

2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


In [ ]:
# stpe 1: tensors are like smart arrays

import torch
x = torch.tensor([2.0, 3.0]) # a 1D tensor(like a list)
print(x)

# this looks like a NumPy array.

tensor([2., 3.])


In [18]:
# step 2: basic math
a = torch.tensor(2.0)
b = torch.tensor(3.0)
c = a + b # still a tensor
print(c)

tensor(5.)


In [20]:
c = a * b
d = a ** 2
e = torch.sin(a)
f = c + d + e
print(f)

tensor(10.9093)


In [21]:
# step 3: let pytorch compute gradients
x = torch.tensor(2.0, requires_grad= True) # track gradients
y = x**2 + 3*x + 1

y.backward() # compute dy/dx

print(x.grad) # will print 7.0 (the derivative at x = 2)

tensor(7.)


In [24]:
# step 4: softmax + cross

# verify the gradient using the torch library
# torch should give you the exact same gradient

logits = torch.tensor([0.0, 3.0, -2.0, 1.0], requires_grad = True)

# softmax
probs = torch.nn.functional.softmax(logits, dim=0)

# negative log likelihood loss : asuume label is class 2
loss = -torch.log(probs[3])

loss.backward()
print("Loss:", loss.item())
print("Gradient:", logits.grad)

Loss: 2.1755154132843018
Gradient: tensor([ 0.0418,  0.8390,  0.0057, -0.8865])
